In [ ]:
## ライブラリのインポート
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np
import time
import random
from qulacs import QuantumState
from qulacs.state import inner_product
from qulacs import QuantumCircuit
from qulacs.gate import to_matrix_gate
from qulacs import QuantumState
from qulacs.gate import Identity, X,Y,Z #パウリ演算子
from qulacs.gate import H
from qulacs.gate import RX,RY,RZ #パウリ演算子についての回転演算
from qulacs.gate import U1,U2,U3 #IBMQの基底ゲート

from fractions import Fraction


## 理想的な回路作成

In [ ]:
# # 係数の絶対値の分布をプロットする関数
# def show_distribution(state, nqubits):
#     plt.bar([i for i in range(pow(2, nqubits))], abs(state.get_vector()))
#     plt.xlabel(f'$2^{nqubits}$個のデータ')
#     plt.ylabel('各状態の確率振幅')
#     plt.ylim(0, 1)
#     plt.show()

# import made_graph as graph

###　動作の確認
初期状態$|0 \cdots 0\rangle$を作成

In [ ]:
# 全体のパラメータ
nqubits = 3
times = 4

In [ ]:
import matplotlib.ticker as ticker
state1 = QuantumState(nqubits)
state1.set_zero_state()

次にアダマールゲートを作成する
$$
H = \frac{1}{\sqrt{2}}(|0\rangle \langle 0 | + |1\rangle \langle 0| + |0\rangle \langle 1| + |1\rangle \langle 1|)
$$

In [ ]:
# def make_Hadamard(nqubits):
#     Hadamard = QuantumCircuit(nqubits)
#     for i in range(nqubits):
#         Hadamard.add_gate(H(i))
#     return  Hadamard
import operation

このアダマールゲートを初期状態$|00 \cdots 00\rangle$に作用させる。
$$
|s\rangle = (H\otimes \cdots \otimes H)|0 \cdots 0 \rangle \\
= \frac{1}{(\sqrt{2})^n}(|0\rangle + |1\rangle)\otimes \cdots \otimes (|0\rangle + |1\rangle) \\
= \frac{1}{(\sqrt{2})^n}(|00 \cdots 00\rangle + |00\cdots 01\rangle + \cdots + |11 \cdots 10\rangle + |11\cdots 11\rangle)|s\rangle \\
= \frac{1}{(\sqrt{2})^n}\sum^{2^n - 1}_{x = 0} |x\rangle
$$

In [ ]:
# Hadamard = make_Hadamard(nqubits)
Hadamard = operation.Hadamard(nqubits)
Hadamard.update_quantum_state(state1)

# show_distribution(state1, nqubits)
import  made_graph as graph
graph.show_distribution(state1, nqubits)

次に解に対する反転操作をする、オラクル$U_w$を作成する。

入力$|x\rangle$に対して$x$が解なら、$-1$を掛けて位相を反転し、解でないなら$1$を掛ける。

$$
\begin{cases}
    U_w|x\rangle = |x\rangle (x \neq w) \\
    U_w|w\rangle = -|w\rangle
  \end{cases}
$$

$$
U_w = I - 2 \sum_{w \in \text{解}} |w\rangle \langle w|
$$

In [ ]:
# オラクルU_wの作成
def make_U_w(nqubits):
    U_w = QuantumCircuit(nqubits)
    CnZ = to_matrix_gate(Z(nqubits-1))
    # i-th qubitが全て1の場合だけゲートを作用
    for i in range(nqubits-1):
        control_index = i
        control_with_value = 1
        CnZ.add_control_qubit(control_index, control_with_value)
    U_w.add_gate(CnZ)
    return U_w

これを作用させると$|s\rangle$
$$
U_w |s\rangle = \frac{1}{(\sqrt{2})^n} \sum_{x = 0, x \neq w}^{2^n - 1} U_w |x\rangle + \frac{1}{(\sqrt{2})^n} U_w |x\rangle \\

= \frac{1}{(\sqrt{2})^n} \sum_{x = 0, x \neq w}^{2^n - 1} |x\rangle - \frac{1}{(\sqrt{2})^n}|w\rangle \\

= \left( |s\rangle - \frac{1}{(\sqrt{2})^n} |w\rangle \right) - \frac{1}{(\sqrt{2})^n} |w\rangle \\

U_w |s\rangle = |s\rangle - \frac{2}{(\sqrt{2})^n} |w\rangle
$$

となる。

In [ ]:
fuga = state1.copy()
U_w = make_U_w(nqubits)
U_w.update_quantum_state(fuga)
# print(fuga.get_vector())
print(fuga)

最後に$|s\rangle$を対象軸にした反転操作$U_s$を定義する

$$
U_s = 2|s\rangle \langle s| - I = H^{\otimes n}(2 |0 \cdots 0\rangle \langle 0 \cdots 0| - I)H^{\otimes n}
$$

$$
\begin{cases}
    U_s |x\rangle = 2\langle s|x|s\rangle - |x\rangle = \frac{2}{\sqrt{2^n}} |x\rangle \\
    U_s |s\rangle = 2\langle s|s|s\rangle - |s\rangle = |s\rangle
 \end{cases}
$$

In [ ]:
# 反転U_sを作る
def make_U_s(nqubits):
    U_s = QuantumCircuit(nqubits)
    for i in range(nqubits):
        U_s.add_gate(H(i))

    ## 2|0><0| - I の実装
    U_s.add_gate(to_matrix_gate(RZ(nqubits-1, 2*np.pi))) ## まず、位相(-1)を全ての状態に付与する。ゲート行列はarrary([[-1,0],[0,-1]])
    U_s.add_gate( X(nqubits-1) )
    ## 全てのi-th qubitが0の場合だけZゲートを作用させる
    CnZ = to_matrix_gate(Z(nqubits-1))
    for i in range(nqubits-1):
        control_index = i
        control_with_value = 0
        CnZ.add_control_qubit(control_index, control_with_value)
    U_s.add_gate( CnZ )
    U_s.add_gate( X(nqubits-1) )

    for i in range(nqubits):
        U_s.add_gate(H(i))

    return U_s

In [ ]:
## 初期状態の準備
state1 = QuantumState(nqubits)
state1.set_zero_state()
Hadamard.update_quantum_state(state1)

## U_s U_w を作用
U_s = make_U_s(nqubits)
U_w.update_quantum_state(state1)
U_s.update_quantum_state(state1)
# show_distribution(state1, nqubits)
graph.show_distribution(state1, nqubits)


In [ ]:
# 何回か繰り返す
## 内積を評価するために 解状態 |1...1> を作っておく
target_state = QuantumState(nqubits)
target_state.set_computational_basis(2**nqubits-1) ## 2**n_qubits-1 は 2進数で 1...1

## グローバーのアルゴリズムの実行
state1 = QuantumState(nqubits)
state1.set_zero_state()

Hadamard.update_quantum_state(state1)

avg_result = []
for i in range(4):
    U_w.update_quantum_state(state1)
    U_s.update_quantum_state(state1)
    # show_distribution(state1, nqubits)
    graph.show_distribution(state1, nqubits)
    fuga = np.linalg.norm(inner_product(state1, target_state))
    avg_result.append(fuga)
    print(fuga)

In [ ]:
## 初期状態の準備
state1 = QuantumState(nqubits)
state1.set_zero_state()
Hadamard.update_quantum_state(state1)

avg_result = []
hoge = np.linalg.norm(inner_product(state1, target_state))
avg_result.append(hoge)
for i in range(times):
    U_w.update_quantum_state(state1)
    U_s.update_quantum_state(state1)
    fuga = np.linalg.norm(inner_product(state1, target_state))
    avg_result.append(fuga)
    print(fuga)

plt.xlabel('オラクルの試行回数k')
plt.ylabel('|11...11>の確率振幅')
plt.ylim(0, 1)
plt.xticks(np.arange(0, len(avg_result), step=1))
plt.grid()
plt.plot(avg_result, "o-")

In [ ]:
def line_graph(result, title):
    plt.title(title)
    plt.plot(result, "o-")
    plt.xlabel('試行回数k')
    plt.ylabel('解の確率振幅')
    plt.ylim(0, 1)
    plt.xticks(np.arange(0, len(result), step=1))
    plt.grid()
    plt.show()


In [ ]:
# グローバーのアルゴリズム
def grover(nqubits, operate_times):
    state = QuantumState(nqubits)
    state.set_zero_state()

    # 内積を評価するために 解状態 |1...1> を作っておく
    target_state = QuantumState(nqubits)
    target_state.set_computational_basis(2 ** nqubits - 1) # 2**n_qubits-1 は 2進数で 1...1

    # グローバーのアルゴリズムの実行
    # Hadamard = make_Hadamard(nqubits)
    Hadamard = operation.Hadamard(nqubits)
    U_w = make_U_w(nqubits)
    U_s = make_U_s(nqubits)

    result = []

    state = QuantumState(nqubits)
    state.set_zero_state()
    Hadamard.update_quantum_state(state)
    hoge = np.linalg.norm(inner_product(state, target_state))
    result.append(hoge)
    for k in range(operate_times):
        U_w.update_quantum_state(state)
        U_s.update_quantum_state(state)
        fuga = np.linalg.norm(inner_product(state, target_state))
        result.append(fuga)
        # print(fuga)


    max_k = np.argmax(result)
    print(f'maximal probability {result[max_k]:5e} is obtained at k = {max_k}')
    return  result, max_k, result[max_k]

In [ ]:
result_array = []
result_theory, k_theory, p_kth= grover(nqubits, times)
line_graph(result_theory, '理想的な回路')

## 任意の回転ゲート
任意の回転ゲートでアダマール演算子を作成する

Qulacsでは任意の回転ゲートを作成する際に、IBMQのOpenQASMを使用する。

以下のコードは、
`U3(対象ビットの添字, \theta, \psi, \lambda)`となっている。
[参考資料](http://docs.qulacs.org/ja/latest/guide/2.0_python_advanced.html#id24)
また、対象ビットの添字はQulacsの仕様上、$|00 \cdots 00\rangle$である場合、一番右から0番目の量子ビットになる
[参考資料](https://docs.qulacs.org/ja/latest/intro/4.1_python_tutorial.html?highlight=%E5%AF%BE%E8%B1%A1%E3%83%93%E3%83%83%E3%83%88%E3%81%AE%E6%B7%BB%E3%81%88%E5%AD%97#id3)

$$
U(\theta, \phi, \lambda) = \begin{pmatrix}
\cos{\frac{\theta}{2}} & -e^{i \lambda} \sin{\frac{\theta}{2}} \\
e^{i \phi}\cos{\frac{\theta}{2}} & e^{i (\lambda + \phi)}\cos{\frac{\theta}{2}}
\end{pmatrix}
$$

In [ ]:
from qulacs.gate import U3
print(U3(0, np.pi/2, 0, np.pi))


### 任意の回転ゲートを利用してアダマールゲートを作成
上記の内容を踏まえてアダマールゲートを作成する。
$$
H = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1 \\
1 & -1
\end{pmatrix}\\
= \begin{pmatrix}
\cos{\frac{\pi}{4}} & -e^{i \pi} \sin{\frac{\pi}{4}} \\
e^{i 0}\cos{\frac{\pi}{4}} & e^{i (\pi + 0)}\cos{\frac{\pi}{4}}
\end{pmatrix} \\
= U(\frac{\pi}{2}, 0, \pi)
$$
これを用いて、以下のように作成する。


In [ ]:
def make_revolution_Hadamard(nqubits):
    U_3 = QuantumCircuit(nqubits)
    THETA = np.pi/2
    PHI = 0
    LAMBDA = np.pi
    # 全てのqubitにゲートを作用
    for i in range(nqubits):
        control_index = i
        U_3.add_gate(U3(control_index, THETA, PHI, LAMBDA))
    return U_3

試しに、通常のアダマール演算子と比較してみる。

In [ ]:
# 通常のアダマール演算子を作成し、 初期化した3量子ビットに作用
# Hadamard = make_Hadamard(3)
Hadamard = operation.Hadamard(3)
state1 = QuantumState(3)
state1.set_zero_state()
Hadamard.update_quantum_state(state1)
print('通常のアダマールゲート')
# show_distribution(state1, 3)
graph.show_distribution(state1, 3)
print(state1.get_vector())

# アダマール演算子と同じ振る舞いをするよう作成した回転ゲートを、
# 初期化した3量子ビットに作用
revolution_Hadamard = make_revolution_Hadamard(3)
state2 = QuantumState(3)
state2.set_zero_state()
revolution_Hadamard.update_quantum_state(state2)
print("作成したアダマールゲート")
# show_distribution(state2, 3)
graph.show_distribution(state2, 3)

print(state2.get_vector())


どのくらいの誤差があるか調べる。

In [ ]:
state1.get_vector() == state2.get_vector()
# 誤差
absolute_error_array = []
relative_error_array = []
for i in range(len(state1.get_vector())):
    absolute_error = state2.get_vector()[i] - state1.get_vector()[i]
    absolute_error_array.append(absolute_error)
    relative_error = absolute_error / state1.get_vector()[i]
    relative_error_array.append(relative_error)

print(f'絶対誤差：{absolute_error_array}')
print(f'相対誤差：{relative_error_array}')


## 任意の回転ゲートを使用してグローバーアルゴリズムを行う。
アダマール演算子と同じ振る舞いをする任意の回転ゲートを使用してグローバーアルゴリズムを行う。

In [ ]:
result = []
def revolution_grover(nqubits, operate_times):
    state = QuantumState(nqubits)
    state.set_zero_state()

    # 内積を評価するために 解状態 |1...1> を作っておく
    target_state = QuantumState(nqubits)
    target_state.set_computational_basis(2 ** nqubits - 1) # 2**n_qubits-1 は 2進数で 1...1

    # グローバーのアルゴリズムの実行
    # Hadamard = make_Hadamard(nqubits)
    Hadamard = make_revolution_Hadamard(nqubits)
    U_w = make_U_w(nqubits)
    U_s = make_U_s(nqubits)

    result = []

    state = QuantumState(nqubits)
    state.set_zero_state()
    Hadamard.update_quantum_state(state)
    hoge = np.linalg.norm(inner_product(state, target_state))
    result.append(hoge)
    for k in range(operate_times):
        U_w.update_quantum_state(state)
        U_s.update_quantum_state(state)
        fuga = np.linalg.norm(inner_product(state, target_state))
        result.append(fuga)
        # print(fuga)


    max_k = np.argmax(result)
    return  result, max_k



In [ ]:
result, max_k = revolution_grover(nqubits, times)
print(result)
line_graph(result, '回転ゲートを使った理想回路')

In [ ]:

grover_result = grover(nqubits, times)
print(grover_result)
print(result)

for i in range(len(grover_result)):
    hoge = abs(result[i] - grover_result[i])
    print(f'絶対誤差：{hoge}')


## 作成した回転ゲートをずらしていく

作成した回転ゲートのy軸をずらしていく。
変数`delta`には$\theta$に加える角度を示す

In [ ]:
def make_noisy_Hadamard(nqubits, delta):
    U_3 = QuantumCircuit(nqubits)
    THETA = np.pi/2
    PHI = 0
    LAMBDA = np.pi
    # 全てのqubitにゲートを作用
    for i in range(nqubits):
        control_index = i
        U_3.add_gate(U3(control_index, THETA + delta, PHI, LAMBDA))
    return U_3

In [ ]:
def noisy_grover(nqubits, operate_times, delta):
    state = QuantumState(nqubits)
    state.set_zero_state()

    # 内積を評価するために 解状態 |1...1> を作っておく
    target_state = QuantumState(nqubits)
    target_state.set_computational_basis(2 ** nqubits - 1) # 2**n_qubits-1 は 2進数で 1...1

    # グローバーのアルゴリズムの実行
    # Hadamard = make_revolution_Hadamard(nqubits)
    Hadamard = make_noisy_Hadamard(nqubits, delta)
    U_w = make_U_w(nqubits)
    U_s = make_U_s(nqubits)

    result = []

    state = QuantumState(nqubits)
    state.set_zero_state()
    Hadamard.update_quantum_state(state)
    hoge = np.linalg.norm(inner_product(state, target_state))
    result.append(hoge)
    for k in range(operate_times):
        U_w.update_quantum_state(state)
        U_s.update_quantum_state(state)
        fuga = np.linalg.norm(inner_product(state, target_state))
        result.append(fuga)
        # print(fuga)

    max_k = np.argmax(result)
    return  result, max_k, result[max_k]


### 3量子ビットで計算
実際に、$\delta = 0$~$2\pi$を$\theta$に加える

In [ ]:
noises = []
noises_array = []
n = 12
for i in range(0, 2*n + 1):
    sample = np.pi * i
    result = sample / n
    noises.append(result)
    noises_array.append(f"{i}π/{n}")

# noises_title = ['0','', '(π/4)',' ',  'π/2', '  ', '3π/4','   ', 'π', '    ', '5π/4', '     ', '3π/2', '      ', '7π/4', '       ', '2π']

print(f'noises:{noises}')

k_k_theory_array = []
p_k_array = []
p_kth_array = []

for i in range(len(noises)):
    result, max_k, p_k = noisy_grover(nqubits, times, noises[i])
    k_k_theory = max_k - k_theory
    p_k = result[max_k]
    p_kth = result[k_theory]

    line_graph(result, f'角度を{noises_array[i]}ズラした回路')
    print(f'maximal probability {result[max_k]:5e} is obtained at k = {max_k}')
    print(f'理想値\k_therorとの差:{k_k_theory}')
    print(f'実験値kの時の確率：{p_k:5e}')
    print(f'理想値k_theoryの時の確率:{p_kth:5e}')
    k_k_theory_array.append(k_k_theory)
    p_k_array.append(p_k)
    p_kth_array.append(p_kth)

In [ ]:
noises = []
noises_array = []
n = 12
for i in range(0, 2*n + 1):
    sample = np.pi * i
    result = sample / n
    noises.append(result)
    noises_array.append(f"{i}π/{n}")

noises_title = ['0', '(π/4)',  'π/2', '3π/4', 'π', '5π/4', '3π/2',  '7π/4',  '2π']
noises_title_int = [0, (np.pi/4),  np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2,  7*np.pi/4,  2*np.pi]

# print(f'noises:{noises}')

k_k_theory_array = []
p_k_array = []
p_kth_array = []

for i in range(len(noises)):
    result, max_k, p_k = noisy_grover(nqubits, times, noises[i])
    k_k_theory = max_k - k_theory
    # k_k_theory = abs(max_k - k_theory)
    p_k = result[max_k]
    p_kth = result[k_theory]
    k_k_theory_array.append(k_k_theory)
    p_k_array.append(p_k)
    p_kth_array.append(p_kth)


#### $k - k_{\text{theory}}$のグラフ化
理想的なグローバーの$U_s$, $U_w$を作用させる回数$k_{\text{theory}}$と、実験で得られてたグローバーの$U_s$, $U_w$を作用させる回数$k$の差が与えたノイズ$\delta_y$でどのように変化するか見てみる。

In [ ]:
def different_of_k_graph(x, y, yticks_start, yticks_end):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xticks(noises_title_int, noises_title)
    ax.set_yticks(np.arange(yticks_start, yticks_end, 1))
    ax.grid(axis = 'both')
    ax.set_xlabel("与えたノイズδ", size = 14, weight = "light")
    ax.set_ylabel('(k) - (k_theory)', size = 14, weight = "light")
    ax.plot(x, y, "-o")

In [ ]:
THREEk_k_theory_array = k_k_theory_array
different_of_k_graph(noises, k_k_theory_array, -3, 3)

#### $P(k)$のグラフ化
ノイズのあるグローバについて、$U_s$, $U_w$を作用させる回数$k$の時の確率がノイズ$\delta_y$でどのように変化するか見てみる

In [ ]:
def P_k_graph(x, y):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xticks(noises_title_int, noises_title)
    ax.set_ylim(0, 1)
    ax.grid(axis = 'both')
    ax.set_xlabel("与えたノイズδ", size = 14, weight = "light")
    ax.set_ylabel("解の確率振幅", size = 14, weight = "light")
    ax.plot(x, y, "-o")

In [ ]:
THREE_p_k_array = p_k_array
P_k_graph(noises, p_k_array)


#### $P(k_{\text{theory})$のグラフ化
ノイズのあるグローバについて、理想的なグローバの$U_s$, $U_w$を作用させる回数$k_{\text{theory}}$の時、確率がノイズ$\delta_y$でどのように変化するのか見てみる


In [ ]:
def P_k_throry_graph(x, y):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xticks(noises_title_int, noises_title)
    ax.set_ylim(0, 1)
    ax.grid(axis = 'both')
    ax.set_xlabel("与えたノイズδ", size = 14, weight = "light")
    ax.set_ylabel("理想値k_{theory}時の解の確率振幅", size = 14, weight = "light")
    ax.plot(x, y, "-o")

In [ ]:
THREE_p_kth_array = p_kth_array
P_k_throry_graph(noises, p_kth_array)

## 5量子ビットで計算

In [ ]:
nqubits = 5
times = 10

In [ ]:
result_array = []
result_theory, k_theory, p_kth= grover(nqubits, times)
line_graph(result_theory, '理想的な回路')

In [ ]:
noises_title = ['0','', '(π/4)',' ',  'π/2', '  ', '3π/4','   ', 'π', '    ', '5π/4', '     ', '3π/2', '      ', '7π/4', '       ', '2π']

print(f'noises:{noises}')

k_k_theory_array = []
p_k_array = []
p_kth_array = []

for i in range(len(noises)):
    result, max_k, p_k = noisy_grover(nqubits, times, noises[i])
    k_k_theory = max_k - k_theory
    p_k = result[max_k]
    p_kth = result[k_theory]

    line_graph(result, f'角度を{noises_array[i]}ズラした回路')
    print(f'maximal probability {result[max_k]:5e} is obtained at k = {max_k}')
    print(f'理想値\k_therorとの差:{k_k_theory}')
    print(f'実験値kの時の確率：{p_k:5e}')
    print(f'理想値k_theoryの時の確率:{p_kth:5e}')
    k_k_theory_array.append(k_k_theory)
    p_k_array.append(p_k)
    p_kth_array.append(p_kth)

In [ ]:
noises = []
noises_array = []
n = 12
for i in range(0, 2*n + 1):
    sample = np.pi * i
    result = sample / n
    noises.append(result)
    noises_array.append(f"{i}π/{n}")

noises_title = ['0', '(π/4)',  'π/2', '3π/4', 'π', '5π/4', '3π/2',  '7π/4',  '2π']
noises_title_int = [0, (np.pi/4),  np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2,  7*np.pi/4,  2*np.pi]

print(f'noises:{noises}')

k_k_theory_array = []
p_k_array = []
p_kth_array = []

for i in range(len(noises)):
    result, max_k, p_k = noisy_grover(nqubits, times, noises[i])
    k_k_theory = max_k - k_theory
    # k_k_theory = abs(max_k - k_theory)
    p_k = result[max_k]
    p_kth = result[k_theory]
    k_k_theory_array.append(k_k_theory)
    p_k_array.append(p_k)
    p_kth_array.append(p_kth)

#### $k - k_{\text{theory}}$のグラフ化
理想的なグローバーの$U_s$, $U_w$を作用させる回数$k_{\text{theory}}$と、実験で得られてたグローバーの$U_s$, $U_w$を作用させる回数$k$の差が与えたノイズ$\delta_y$でどのように変化するか見てみる。

In [ ]:
FIVE_k_k_theory_array = k_k_theory_array
different_of_k_graph(noises, k_k_theory_array, -5, 7)
# different_of_k_graph(noises, k_k_theory_array, 8, 4)

#### $P(k)$のグラフ化
ノイズのあるグローバについて、$U_s$, $U_w$を作用させる回数$k$の時の確率がノイズ$\delta_y$でどのように変化するか見てみる

In [ ]:
FIVE_p_k_array = p_k_array
P_k_graph(noises, p_k_array)

#### $P(k_{\text{theory})$のグラフ化
ノイズのあるグローバについて、理想的なグローバの$U_s$, $U_w$を作用させる回数$k_{\text{theory}}$の時、確率がノイズ$\delta_y$でどのように変化するのか見てみる

In [ ]:
FIVE_p_kth_array = p_kth_array
P_k_throry_graph(noises, p_kth_array)

## 10量子ビットで計算

In [ ]:
nqubits = 10
times = 27

In [ ]:
result_array = []
result_theory, k_theory, p_kth= grover(nqubits, times)
line_graph(result_theory, '理想的な回路')

In [ ]:
noises_title = ['0','', '(π/4)',' ',  'π/2', '  ', '3π/4','   ', 'π', '    ', '5π/4', '     ', '3π/2', '      ', '7π/4', '       ', '2π']

print(f'noises:{noises}')

k_k_theory_array = []
p_k_array = []
p_kth_array = []

for i in range(len(noises)):
    result, max_k, p_k = noisy_grover(nqubits, times, noises[i])
    k_k_theory = max_k - k_theory
    p_k = result[max_k]
    p_kth = result[k_theory]

    line_graph(result, f'角度を{noises_array[i]}ズラした回路')
    print(f'maximal probability {result[max_k]:5e} is obtained at k = {max_k}')
    print(f'理想値\k_therorとの差:{k_k_theory}')
    print(f'実験値kの時の確率：{p_k:5e}')
    print(f'理想値k_theoryの時の確率:{p_kth:5e}')
    k_k_theory_array.append(k_k_theory)
    p_k_array.append(p_k)
    p_kth_array.append(p_kth)

In [ ]:
noises = []
noises_array = []
n = 12
for i in range(0, 2*n + 1):
    sample = np.pi * i
    result = sample / n
    noises.append(result)
    noises_array.append(f"{i}π/{n}")

noises_title = ['0', '(π/4)',  'π/2', '3π/4', 'π', '5π/4', '3π/2',  '7π/4',  '2π']
noises_title_int = [0, (np.pi/4),  np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2,  7*np.pi/4,  2*np.pi]

print(f'noises:{noises}')

k_k_theory_array = []
p_k_array = []
p_kth_array = []

for i in range(len(noises)):
    result, max_k, p_k = noisy_grover(nqubits, times, noises[i])
    k_k_theory = max_k - k_theory
    p_k = result[max_k]
    p_kth = result[k_theory]
    k_k_theory_array.append(k_k_theory)
    p_k_array.append(p_k)
    p_kth_array.append(p_kth)

#### $k - k_{\text{theory}}$のグラフ化
理想的なグローバーの$U_s$, $U_w$を作用させる回数$k_{\text{theory}}$と、実験で得られてたグローバーの$U_s$, $U_w$を作用させる回数$k$の差が与えたノイズ$\delta_y$でどのように変化するか見てみる。

In [ ]:
TEN_k_k_theory_array = k_k_theory_array
different_of_k_graph(noises, k_k_theory_array, -27, 1)

#### $P(k)$のグラフ化
ノイズのあるグローバについて、$U_s$, $U_w$を作用させる回数$k$の時の確率がノイズ$\delta_y$でどのように変化するか見てみる

In [ ]:
TEN_p_k_array = p_k_array
P_k_graph(noises, p_k_array)

#### $P(k_{\text{theory})$のグラフ化
ノイズのあるグローバについて、理想的なグローバの$U_s$, $U_w$を作用させる回数$k_{\text{theory}}$の時、確率がノイズ$\delta_y$でどのように変化するのか見てみる

In [ ]:
TEN_p_kth_array = p_kth_array
P_k_throry_graph(noises, p_kth_array)

## グラフを重ねてみる

#### $k - k_{\text{theory}}$のグラフを重ねてみる

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x = noises
ax.set_xticks(noises_title_int, noises_title)
ax.set_yticks(np.arange(-27, 8, 1))
ax.grid(axis = 'both')
ax.set_xlabel("与えたノイズδ", size = 14, weight = "light")
ax.set_ylabel("(k) - (k_theory)", size = 14, weight = "light")
l1, l2, l3 = "3量子ビット", "5量子ビット", "10量子ビット"
ax.plot(x, THREEk_k_theory_array, "-o", label = l1)
ax.plot(x, FIVE_k_k_theory_array, "-+", label = l2)
ax.plot(x, TEN_k_k_theory_array, "-x", label = l3)
ax.legend(loc = 0)

#### $P(k)$のグラフ化

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x = noises
ax.set_xticks(noises_title_int, noises_title)
ax.grid(axis = 'both')
ax.set_xlabel("与えたノイズδ", size = 14, weight = "light")
ax.set_ylabel("実験値kの時の確率", size = 14, weight = "light")
l1, l2, l3 = "3量子ビット", "5量子ビット", "10量子ビット"
ax.plot(x, THREE_p_k_array, "-o", label = l1)
ax.plot(x, FIVE_p_k_array, "-+", label = l2)
ax.plot(x, TEN_p_k_array, "-x", label = l3)
ax.legend(loc = 0)

#### $P(k_{\text{\theta})$のグラフ化

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x = noises
ax.set_xticks(noises_title_int, noises_title)
ax.grid(axis = 'both')
ax.set_xlabel("与えたノイズδ", size = 14, weight = "light")
ax.set_ylabel("理想値k_theoryの時の確率", size = 14, weight = "light")
l1, l2, l3 = "3量子ビット", "5量子ビット", "10量子ビット"
ax.plot(x, THREE_p_kth_array, "-o", label = l1)
ax.plot(x, FIVE_p_kth_array, "-+", label = l2)
ax.plot(x, TEN_p_kth_array, "-x", label = l3)
ax.legend(loc = 0)


## $\frac{\pi}{2}$とその直前直後のグラフ
$\frac{\pi}{2}$とその直前、直後のグラフを作成し、その違いを見てみる

In [ ]:
nqubits = 3
times = 5

result_array = []
result_theory, k_theory, p_kth= grover(nqubits, times)
line_graph(result_theory, '理想的な回路')

In [ ]:
noises = []
noises_array = []
n = 24
for i in range(10, 14 + 1):
    sample = np.pi * i
    result = sample / n
    noises.append(result)
    noises_array.append(f"{i}π/{n}")

for i in range(len(noises)):
    result, max_k, p_k = noisy_grover(nqubits, times, noises[i])
    k_k_theory = max_k - k_theory
    # p_k = result[max_k]
    # p_kth = result[k_theory]

    line_graph(result, f'角度を{noises_array[i]}ズラした回路')
    print(f'maximal probability {result[max_k]:5e} is obtained at k = {max_k}')
    print(f'理想値\k_therorとの差:{k_k_theory}')
    print(f'実験値kの時の確率：{p_k:5e}')
    print(f'理想値k_theoryの時の確率:{p_kth:5e}')
    k_k_theory_array.append(k_k_theory)
    p_k_array.append(p_k)
    p_kth_array.append(p_kth)

## δ = πの時の状態

In [ ]:
## 初期状態の準備
state1 = QuantumState(nqubits)
state1.set_zero_state()
Hadamard = make_noisy_Hadamard(3, np.pi)
Hadamard.update_quantum_state(state1)
# show_distribution(state1, nqubits)
graph.show_distribution(state1, nqubits)

## U_s U_w を作用
U_s = make_U_s(nqubits)
U_w.update_quantum_state(state1)
U_s.update_quantum_state(state1)
# show_distribution(state1, nqubits)
graph.show_distribution(state1, nqubits)

for _ in range(4):
    U_s = make_U_s(nqubits)
    U_w.update_quantum_state(state1)
    U_s.update_quantum_state(state1)
    # show_distribution(state1, nqubits)
    graph.show_distribution(state1, nqubits)
